In [1]:
from datetime import datetime
import requests

URL2007 =  'https://gml.noaa.gov/ccl/getresultsx2007.php'  # NOAA GML WMO X2007 Scale API URL
URL2019 =  'https://gml.noaa.gov/ccl/getresults.php'  # NOAA GML WMO X2019 Scale API URL. May need to be updated in the future if a new scale comes out.

class WMOSCALE():
    def __init__(self, url):
        self._url = url

    def get_all_cal_info(self, sn: str, gas: str) -> list:
        params = {'sn': sn.upper(),
                  'gas': gas.lower(),
                  'format': 'json'}
        response = requests.get(self._url, params = params)
        if response.status_code == requests.codes.ok:
            cal_data = response.json()
            data = []
            for cal in cal_data:
                _cal = {}  #Reformat and rename the data.
                _cal['sn'] = cal['serial_number']
                dt = datetime.strptime(' '.join((cal['date'], cal['time'])), '%Y-%m-%d %H:%M:%S')
                dt_str = dt.strftime('%Y-%m-%dT%H:%M:%SZ')
                _cal['datetime'] = dt_str
                _cal['fill_code'] = cal['fill_code']
                _cal['parameter'] = cal['parameter']
                _cal['value'] = cal['value']
                _cal['sd'] = cal['std._dev.']
                _cal['num_samples'] = cal['num_samples']
                _cal['cal_type'] = cal['type']
                _cal['cal_system'] = cal['system']
                _cal['cal_instrument_code'] = cal['instrument_code']
                _cal['tank_pressure'] = cal['tank_pressure']
                _cal['flag'] = cal['flag']
                _cal['scale'] = cal['scale']
                _cal['uncertainty'] = cal['uncertainty']
                data.append(_cal)
            return data
        else:
            raise ConnectionError(f"{response.status_code}: {response.reason}")
            
    def get_recent_cal_info(self, sn: str, gas: str) -> dict:
        cal_data = self.get_all_cal_info(sn, gas)
        dts = [datetime.strptime(v['datetime'], '%Y-%m-%dT%H:%M:%SZ') for v in cal_data]
        recent_dt = max(dts)
        [recent_cal] = [v for v in cal_data if v['datetime'] == recent_dt.strftime('%Y-%m-%dT%H:%M:%SZ')]
        return recent_cal
            
class X2007(WMOSCALE):
    def __init__(self) -> None:
        super().__init__(URL2007)
            
class X2019(WMOSCALE):
    def __init__(self) -> None:
        super().__init__(URL2019)


In [2]:
%%time
x2019 = X2019()
gas = 'co2'
std1 = x2019.get_recent_cal_info(sn = 'CB11612', gas = gas)
std2 = x2019.get_recent_cal_info(sn = 'CC309309', gas = gas)
std3 = x2019.get_recent_cal_info(sn = 'CA02044', gas = gas)
std4 = x2019.get_recent_cal_info(sn = 'CA06653', gas = gas)
std5 = x2019.get_recent_cal_info(sn = 'CA05660', gas = gas)
for std in [std1, std2, std3, std4, std5]:
    print(std)

{'sn': 'CB11612', 'datetime': '2017-04-06T11:42:00Z', 'fill_code': 'B', 'parameter': 'CO2', 'value': 451.23, 'sd': 0.011, 'num_samples': 8, 'cal_type': 'CRDS', 'cal_system': 'co2cal-2', 'cal_instrument_code': 'PC1', 'tank_pressure': 2015, 'flag': '.', 'scale': 'CO2_X2019', 'uncertainty': 0.01}
{'sn': 'CC309309', 'datetime': '2013-12-16T11:25:00Z', 'fill_code': 'B', 'parameter': 'CO2', 'value': 107.104, 'sd': 0.027, 'num_samples': 3, 'cal_type': 'NDIR', 'cal_system': 'co2cal-1', 'cal_instrument_code': 'L9', 'tank_pressure': 2000, 'flag': '.', 'scale': 'CO2_X2019', 'uncertainty': -99.9}
{'sn': 'CA02044', 'datetime': '2015-04-21T14:02:00Z', 'fill_code': 'F', 'parameter': 'CO2', 'value': 504.047, 'sd': 0.026, 'num_samples': 3, 'cal_type': 'NDIR', 'cal_system': 'co2cal-1', 'cal_instrument_code': 'L9', 'tank_pressure': 1950, 'flag': '.', 'scale': 'CO2_X2019', 'uncertainty': 0.03}
{'sn': 'CA06653', 'datetime': '2013-06-07T23:03:00Z', 'fill_code': 'A', 'parameter': 'CO2', 'value': 257.1, 'sd':